In [23]:
import polars as pl 

# Explore Metadata

In [24]:
metadata_df = pl.read_csv("../data/adex-database/metadata.tsv", separator="\t")

In [25]:
metadata_df

Sample,GSE,Experimental Strategy,GPL,Condition,Tissue,Cell Type,Gender,Age,Ethnicity
str,str,str,str,str,str,str,str,str,str
"""GSM260887""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260890""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260892""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260895""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260898""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260901""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260904""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260907""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260910""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""


In [29]:
total_samples = metadata_df.select("Sample").shape[0]
unique_samples = metadata_df.select("Sample").unique().shape[0]

conditions_df = metadata_df.select("Condition").unique()
conditions_count = conditions_df.shape[0]
conditions = conditions_df.to_series().to_list()

tissues_df = metadata_df.select("Tissue").unique()
tissues_count = tissues_df.shape[0]
tissues = tissues_df.to_series().to_list()

gpl_df = metadata_df.select("GPL").unique()
gpl_count = gpl_df.shape[0]
gpls = gpl_df.to_series().to_list()

males = metadata_df.select("Gender").filter(pl.col("Gender") == "Male").shape[0]
females = metadata_df.select("Gender").filter(pl.col("Gender") == "Female").shape[0]
gender_unknown = metadata_df.select("Gender").filter((pl.col("Gender") != "Female") & (pl.col("Gender") != "Male")).shape[0]

print(f"""
Total Samples = {total_samples}
Unique Samples = {unique_samples}
Conditions = {conditions_count} [{",".join(conditions)}]
GPL = {gpl_count} [{",".join(gpls)}]
Tissues = {tissues_count} [{",".join(tissues)}]
Males = {males}, Females = {females}, Unknown Gender = {gender_unknown}
""")


Total Samples = 3980
Unique Samples = 3854
Conditions = 6 [SjS,SSc,SLE,T1D,Healthy,RA]
GPL = 13 [GPL13158,GPL10558,GPL570,GPL96,GPL97,GPL15456,GPL10558_GPL13534,GPL20301,GPL6884,GPL11154,GPL16791,GPL14951_GPL13534,GPL13667]
Tissues = 7 [Parotid gland,Salivary gland,Saliva,Whole blood,Synovial membrane,Skin,Peripheral blood]
Males = 269, Females = 1547, Unknown Gender = 2164


In [27]:
metadata_df.group_by("Condition").agg(pl.struct(["Sample"]).n_unique().alias('SamplesPerCondition')).sort("SamplesPerCondition", descending=True)

Condition,SamplesPerCondition
str,u32
"""SLE""",1502
"""Healthy""",957
"""RA""",862
"""SjS""",281
"""SSc""",157
"""T1D""",95


In [28]:
metadata_df.group_by("Age").agg(pl.struct(["Sample"]).n_unique().alias('SamplesPerAge')).sort("Age")


Age,SamplesPerAge
str,u32
"""1-10""",113
"""11-20""",342
"""21-30""",383
"""31-40""",331
"""41-50""",105
"""51-60""",66
"""61-70""",69
"""71-80""",39
""">80""",6
